In [1]:
%defaultDatasource jdbc:h2:mem:db

In [ ]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;
DROP TABLE IF EXISTS SideEffect;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../../data/faers-2017/sideeffect.csv');

In [5]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [ ]:
SELECT a.codeDrug, b.codeDrug, COUNT(*) intCount
FROM DrugUse a, DrugUse b
WHERE a.idPerson = b.idPerson AND a.codeDrug < b.codeDrug
GROUP BY a.codeDrug, b.codeDrug
HAVING intCount > 30

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [10]:
DROP TABLE IF EXISTS DrugInteraction;

CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../../data/faers-2017/results/drug-interaction.csv');

SELECT * FROM DrugInteraction;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [5]:
SELECT druga, drugb, CAST(weight AS DOUBLE) / CAST((SELECT SUM(weight) FROM DrugInteraction) AS DOUBLE) AS Suporte
FROM DrugInteraction
WHERE weight > 30
ORDER BY Suporte

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [6]:
CREATE VIEW teste AS
SELECT druga AS drug, SUM(weight) drugCount
FROM DrugInteraction
GROUP BY druga
UNION ALL
SELECT drugb AS drug, SUM(weight) drugCount
FROM DrugInteraction
GROUP BY drugb

In [7]:
CREATE VIEW counterDrug AS SELECT drug, SUM(drugCount) drugCount FROM teste GROUP BY drug ORDER BY drug

In [17]:
CREATE VIEW respostaParcial AS
-- drugA => drubB
SELECT DI.druga AS A, DI.drugb AS B, CAST(DI.weight AS DOUBLE) / CAST(CD.drugCount AS DOUBLE) Confianca
FROM DrugInteraction DI, counterDrug CD
WHERE weight > 30 AND DI.druga = CD.drug
--
UNION ALL
-- drugB => drubA
SELECT DI.drugb AS A, DI.druga AS B, CAST(DI.weight AS DOUBLE) / CAST(CD.drugCount AS DOUBLE) Confianca
FROM DrugInteraction DI, counterDrug CD
WHERE weight > 30 AND DI.drugb = CD.drug

In [19]:
SELECT * FROM respostaParcial ORDER BY Confianca

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [6]:
CREATE VIEW contagemDrogas AS
SELECT codeDrug, COUNT(*) total
FROM DrugUse
GROUP BY codeDrug
ORDER BY codeDrug

In [7]:
CREATE VIEW pessoasAfetadas AS
SELECT idPerson, codePathology, codeDrug
FROM DrugUse 
WHERE idPerson = ANY (SELECT idPerson FROM SideEffect)

In [8]:
CREATE VIEW contagemEfeitosColaterais AS
SELECT codeDrug, COUNT(*) afetados
FROM pessoasAfetadas 
GROUP BY codeDrug
ORDER BY codeDrug

In [9]:
-- ratio of people who uses one drug and have some kind of side effect
SELECT CEC.codeDrug, CAST(CEC.afetados AS DOUBLE) / CAST(CD.total AS DOUBLE) afetados
FROM contagemEfeitosColaterais CEC, contagemDrogas CD
WHERE CEC.codeDrug = CD.codeDrug

In [ ]:
-- get the number of people who consumes drug A and drug B and have some 
-- kind of side effect
CREATE VIEW drugCombinationWithSideEffects AS
SELECT a.codeDrug A, b.codeDrug B, COUNT(*) affected
FROM SideEffect SE, DrugUse a, DrugUse b
WHERE SE.idPerson = a.idPerson AND a.idPerson = b.idPerson AND
      a.codeDrug < b.codeDrug
GROUP BY a.codeDrug, b.codeDrug

In [ ]:
-- ratio of people who uses drug A and drug B and have some kind of side effect
SELECT DCWSE.A, DCWSE.B, CAST(DCWSE.affected AS DOUBLE) / CAST(DI.weight AS DOUBLE)
FROM drugCombinationWithSideEffects DCWSE, drugInteraction DI
WHERE DCWSE.A = DI.druga AND DCWSE.B = DI.grub 

In [11]:
SELECT * FROM SideEffect